In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install torchmetrics[detection]

In [ ]:
import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt
import os
from tqdm import tqdm
import torch
from torchmetrics.detection.mean_ap import MeanAveragePrecision
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

In [ ]:
!git clone https://github.com/Pseudo-Lab/Tutorial-Book-Utils
!python Tutorial-Book-Utils/PL_data_loader.py --data FaceMaskDetection
!unzip -q Face\ Mask\ Detection.zip

In [ ]:
%cd Tutorial-Book-Utils/
import utils_ObjectDetection as utils

In [ ]:
!pip install tensorboardX

In [ ]:
from tensorboardX import SummaryWriter
summary = SummaryWriter()

In [ ]:
box = pd.read_csv('/content/drive/MyDrive/TrafficSignDataSet(1)/train/_annotations.csv')
box

In [ ]:
sample = cv2.imread('/content/drive/MyDrive/TrafficSignDataSet(1)/train/img192_jpg.rf.577ecc14027f3fd5b42147bc7845f365.jpg')
sample = cv2.cvtColor(sample, cv2.COLOR_BGR2RGB)
point = box.iloc[0]
pt1 = (int(point['xmin']), int(point['ymax']))
pt2 = (int(point['xmax']), int(point['ymin']))
cv2.rectangle(sample, pt1, pt2, color=(255,0,0), thickness=5)
plt.imshow(sample)

In [ ]:
class TrafficSignDataset(Dataset):
    def __init__(self, df, image_dir, transforms=None):
        super().__init__()
        
        self.image_ids = df["image"].unique() # all image filenames
        self.df = df
        self.image_dir = image_dir # dir to image files
        self.transforms = transforms

    def __getitem__(self, idx: int):
        image_id = self.image_ids[idx]
        records = self.df[self.df["image"] == image_id]
        image = cv2.imread(f"{self.image_dir}/{image_id}", cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32)
        image /= 255.0
        image = torch.tensor(image)
        image = image.permute(2,0,1)
        
        
        boxes = records[["xmin", "ymin", "xmax", "ymax"]].values
        
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
        area = torch.as_tensor(area, dtype=torch.float32)
        
        # class가 1종류이기 때문에 label은 1로만 지정
        labels = torch.as_tensor((records["class"]).values, dtype=torch.int64)

        
        target = {}
        target["boxes"] = torch.tensor(boxes)
        target["labels"] = labels
        target["image_id"] = torch.tensor([idx])
        target["area"] = area


        if self.transforms:
            sample = {"image": image, "boxes": target["boxes"], "labels": labels}
            sample = self.transforms(**sample)
            image = sample["image"]
            target["boxes"] = torch.stack(tuple(map(torch.tensor, zip(*sample["boxes"])))).permute(1, 0)

        return image, target, image_id

    def __len__(self):
        return self.image_ids.shape[0]

In [ ]:
def collate_fn(batch):
    return tuple(zip(*batch))

dir_train = "/content/drive/MyDrive/TrafficSignDataSet(1)/train"
train_ds = TrafficSignDataset(box, dir_train)

train_dl = DataLoader(train_ds, batch_size=8, shuffle=False, num_workers=2, collate_fn=collate_fn)


In [ ]:
def make_prediction(model, img, threshold):
    model.eval()
    preds = model(img)
    for id in range(len(preds)) :
        idx_list = []

        for idx, score in enumerate(preds[id]['scores']) :
            if score > threshold : 
                idx_list.append(idx)

        preds[id]['boxes'] = preds[id]['boxes'][idx_list]
        preds[id]['labels'] = preds[id]['labels'][idx_list]
        preds[id]['scores'] = preds[id]['scores'][idx_list]

    return preds

In [ ]:
test_box = pd.read_csv("/content/drive/MyDrive/TrafficSignDataSet(1)/test/_annotations.csv")
test_box

In [ ]:
dir_test = "/content/drive/MyDrive/TrafficSignDataSet(1)/test"
test_ds = TrafficSignDataset(test_box, dir_test)
test_dl = DataLoader(test_ds, batch_size=8, collate_fn=collate_fn)

In [ ]:
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection import f


In [ ]:
# Load model pretrained on COCO
model = fasterrcnn_resnet50_fpn(pretrained=True)

In [ ]:
num_classes = 4 # 1 class (car) + background

# get number of input features for the classifier
in_features = model.roi_heads.box_predictor.cls_score.in_features

# replace pre-trained head with new one
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

print(model)

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [ ]:
model.to(device)
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.Adam(params, lr=0.0005, weight_decay=0.0005)

In [ ]:
import time

model.train()

num_epochs = 4

start_time = time.perf_counter()
for epoch in range(num_epochs):

    for i, (images, targets, image_ids) in enumerate(train_dl):
        optimizer.zero_grad()
        images = list(image.to(device) for image in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
            
        loss_dict = model(images, targets)

        losses = sum(loss for loss in loss_dict.values())

        losses.backward()
        optimizer.step()

        
        summary.add_scalar('loss/loss_box_reg', loss_dict["loss_box_reg"], epoch)
        summary.add_scalar('loss/loss_classifier', loss_dict["loss_classifier"], epoch)
        summary.add_scalar('loss/loss_objectness', loss_dict["loss_objectness"], epoch)

        summary.add_scalar('loss/Total', losses, epoch)

        if (i+1) % 10 == 0:
            print(f'Epoch {epoch+1} - Total: {losses:.4f}, Regression: {loss_dict["loss_box_reg"]:.4f}, Classifier: {loss_dict["loss_classifier"]:.4f}, Objectness: {loss_dict["loss_objectness"]:.4f}')
end_time = time.perf_counter()
print("실행시간 :",(end_time-start_time))
        

In [ ]:
preds = [dict(boxes=torch.tensor([[258.0, 41.0, 606.0, 285.0]]), scores=torch.tensor([0.536]), labels=torch.tensor([0]))]
target = [dict(boxes=torch.tensor([[214.0, 41.0, 562.0, 285.0]]), labels=torch.tensor([0]))]

In [ ]:

%load_ext tensorboard
%tensorboard --logdir runs

In [ ]:
def make_prediction(model, img, threshold):
    model.eval()
    preds = model(img)
    for id in range(len(preds)) :
        idx_list = []

        for idx, score in enumerate(preds[id]['scores']) :
            if score > threshold : 
                idx_list.append(idx)

        preds[id]['boxes'] = preds[id]['boxes'][idx_list]
        preds[id]['labels'] = preds[id]['labels'][idx_list]
        preds[id]['scores'] = preds[id]['scores'][idx_list]

    return preds

In [ ]:
print(test_dl)

In [ ]:
from tqdm import tqdm

labels = []
preds_adj_all = []
annot_all = []

for im, annot, image_id in test_dl:
    im = list(img.to(device) for img in im)
    #annot = [{k: v.to(device) for k, v in t.items()} for t in annot]

    for t in annot:
        labels += t['labels']

    with torch.no_grad():
        preds_adj = make_prediction(model, im, 0.5)
        preds_adj = [{k: v.to(torch.device('cpu')) for k, v in t.items()} for t in preds_adj]
        preds_adj_all.append(preds_adj)
        annot_all.append(annot)


In [ ]:
%cd Tutorial-Book-Utils/
import utils_ObjectDetection as utils

In [ ]:
sample_metrics = []
for batch_i in range(len(preds_adj_all)):
    sample_metrics += utils.get_batch_statistics(preds_adj_all[batch_i], annot_all[batch_i], iou_threshold=0.5) 

true_positives, pred_scores, pred_labels = [torch.cat(x, 0) for x in list(zip(*sample_metrics))]  # 배치가 전부 합쳐짐
precision, recall, AP, f1, ap_class = utils.ap_per_class(true_positives, pred_scores, pred_labels, torch.tensor(labels))
mAP = torch.mean(AP)
print(f'mAP : {mAP}')
print(f'AP : {AP}')

In [ ]:
images = cv2.imread("/content/drive/MyDrive/TrafficSignDataSet(1)/test/img252_jpg.rf.5ffedb6f063ebff3bb22643d5ec5d489.jpg", cv2.IMREAD_COLOR)
images = cv2.cvtColor(images, cv2.COLOR_BGR2RGB).astype(np.float32)
images /= 255.0
sample = images
images = torch.tensor(images)
images = images.permute(2,0,1)
images = torch.unsqueeze(images, 0)
images = images.to(device)
model.eval()
cpu_device = torch.device("cpu")

outputs = model(images)
outputs = [{k: v.to(cpu_device) for k, v in t.items()} for t in outputs]
mask = outputs[0]['scores'] > 0.7
boxes = outputs[0]["boxes"][mask].detach().numpy().astype(np.int32)
label = outputs[0]["labels"][mask].detach().numpy().astype(np.int32)

In [ ]:
print(outputs[0]['scores']>0.7)
print(mask)
print(outputs[0]["boxes"][mask].detach().numpy().astype(np.int32))
print(boxes)
for i, box in enumerate(boxes):

    if label[i] == 1:
        color = (255, 0, 0)
    elif label[i] == 2:
        color = (0, 255, 0)
    elif label[i] == 3:
        color = (0, 0, 255)

    cv2.rectangle(sample, 
                  (box[0], box[1]), 
                  (box[2], box[3]), 
                  color, 4)
    
plt.imshow(sample)

In [ ]:
import gc
gc.collect()
torch.cuda.empty_cache()